In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_pickle("results.pkl")

In [ ]:
df.columns

In [ ]:
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x))
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x))
df.sort_values(["experiment_type", "Trial Number"])
df["Nodes"] = df["Nodes"].astype(str)

In [ ]:
# df.to_csv("results.csv")

In [ ]:
fig = px.bar(df[df["experiment_type"] == "Single"], x="Nodes", 
                y="Time Taken", 
                barmode="group", 
                color="Success", 
                facet_col="perturber_class", 
                facet_row="top_k", 
                pattern_shape="Status", 
                hover_data=["top_k", "Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations"],
                category_orders={"experiment_type": ["Single", "Sets", "Multiple Pairs"], 
                                "Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                "Success": [True, False]}
            )
fig.update_xaxes(visible=False)
fig.show()